In [ ]:
!pip install -q pystan==2.19.1.1
!pip install -q prophet
!pip install -q greykite

In [ ]:
from collections import defaultdict
import warnings

warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import plotly

from greykite.common.data_loader import DataLoader
from greykite.framework.templates.autogen.forecast_config import ForecastConfig
from greykite.framework.templates.autogen.forecast_config import MetadataParam
from greykite.framework.templates.autogen.forecast_config import ModelComponentsParam
from greykite.framework.templates.forecaster import Forecaster
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.framework.utils.result_summary import summarize_grid_search_results
from greykite.framework.input.univariate_time_series import UnivariateTimeSeries

In [ ]:
df = pd.read_csv('data_principal_covid.csv')

In [ ]:
df.DIA = df.DIA.astype('datetime64[ns]')

In [ ]:
df = df[['DIA', 'POSITIVOS_DIA']]
value_col = 'POSITIVOS_DIA'
time_col="DIA"

In [ ]:
# EDA
ts = UnivariateTimeSeries()
ts.load_data(
    df=df,
    time_col=time_col,
    value_col=value_col,
    freq="D")

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


In [ ]:
print(ts.time_stats)         # time statistics
print(ts.value_stats)        # value statistics
print(ts.freq)               # frequency
print(ts.regressor_cols)     # available regressors
#print(ts.last_date_for_fit)  # last date with value_col
print(ts.last_date_for_reg)  # last date for any regressor
print(ts.df.head())          # the standardized dataset for forecasting
print(ts.fit_df.head())      # the standardized dataset for fitting and historical evaluation

{'gaps':   right_before_gap right_after_gap  gap_size
0       2020-01-05      2020-03-11      65.0
1       2020-03-11      2020-03-13       1.0
2       2020-03-13      2020-03-15       1.0
3       2020-03-15      2020-03-19       3.0
4       2020-03-20      2020-03-23       2.0
5       2020-04-02      2020-04-04       1.0
6       2020-04-04      2020-04-06       1.0
7       2020-04-11      2020-04-13       1.0
8       2020-05-17      2020-05-19       1.0, 'added_timepoints': 76, 'dropped_timepoints': 0, 'data_points': 505, 'mean_increment_secs': 86400.0, 'min_timestamp': Timestamp('2020-01-05 00:00:00'), 'max_timestamp': Timestamp('2021-05-23 00:00:00')}
count    429.000000
mean     126.962704
std      114.207319
min        1.000000
25%       23.000000
50%      114.000000
75%      193.000000
max      629.000000
Name: y, dtype: float64
D
[]
None
                   ts    y
2020-01-05 2020-01-05  1.0
2020-01-06 2020-01-06  NaN
2020-01-07 2020-01-07  NaN
2020-01-08 2020-01-08  NaN
2020-01-

In [ ]:
df

,DIA,POSITIVOS_DIA
0,2020-01-05,1
1,2020-03-11,2
2,2020-03-13,1
3,2020-03-15,1
4,2020-03-19,3
...,...,...
424,2021-05-19,213
425,2021-05-20,200
426,2021-05-21,167
427,2021-05-22,115


In [ ]:
fig = ts.plot()
plotly.io.show(fig)
fig.show(renderer="colab")

In [ ]:
fig = ts.plot_grouping_evaluation(
    aggregation_func=np.mean,  # any aggregation function you want
    aggregation_func_name="mean",
    groupby_time_feature=None,
    groupby_sliding_window_size=7,  # any aggregation window you want
                                    # (7*24 for weekly aggregation of hourly data)
    groupby_custom_column=None,
    title=f"Weekly average of {value_col}")
plotly.io.show(fig)
fig.show(renderer="colab")

In [ ]:
fig = ts.plot_grouping_evaluation(
    aggregation_func=np.mean,
    aggregation_func_name="mean",
    groupby_time_feature="str_dow",  #str_dow, str_doy
    groupby_sliding_window_size=None,
    groupby_custom_column=None,
    title=f"weekly seasonality: mean of {value_col}")
plotly.io.show(fig)
fig.show(renderer="colab")

In [ ]:
fig = ts.plot_grouping_evaluation(
    aggregation_func=np.mean,
    aggregation_func_name="mean",
    groupby_time_feature="year_month",  # year, year_woy, year_woy_dow, year_month
    groupby_sliding_window_size=None,
    groupby_custom_column=None,
    title=f"yearly seasonality: mean of {value_col}")
plotly.io.show(fig)
fig.show(renderer="colab")

In [ ]:
fig = ts.plot_quantiles_and_overlays(groupby_time_feature="year_woy",
                               show_mean=True,
                               show_quantiles=True,
                                show_overlays=True,
                                overlay_style={"line": {"width": 1}, "opacity": 0.5},
                                xlabel="month of year",
                                ylabel='POSITIVOS_DIA',
                                center_values=False,
                                title="yearly seasonality for each year (centered)",
                                )
plotly.io.show(fig)
fig.show(renderer="colab")

In [ ]:
fig = ts.plot_quantiles_and_overlays(
    groupby_time_feature="dow",            # Rows: a row for each day of week (1, 2, ..., 7)
    show_mean=True,                        # mean value on that day
    show_quantiles=[0.1, 0.5, 0.9],        # quantiles of the observed distribution on that day
    show_overlays=True,                    # Include overlays defined by ``overlay_label_time_feature``
    overlay_label_sliding_window_size=90,  # One column for each 90 period sliding window in the dataset,
    aggfunc="median")  
plotly.io.show(fig)
fig.show(renderer="colab")

## Puntos de cambio de tendencia y periocidad

In [ ]:
from greykite.algo.changepoint.adalasso.changepoint_detector import ChangepointDetector

In [ ]:
model = ChangepointDetector()
res = model.find_trend_changepoints(
     df=df,            # data df
     time_col="DIA",    # time column name
     value_col="POSITIVOS_DIA",
     adaptive_lasso_initial_estimator='ridge', # ols, ridge and lasso 
     regularization_strength=0.4,# 0 - 1
     resample_freq="2D",
     no_changepoint_proportion_from_end=0.1) # no saque cambios en el ultimo 20 porciento de la info

fig = model.plot(plot=False,# plot = False returns a plotly figure object.
                 )  
plotly.io.show(fig)
fig.show(renderer="colab")

In [ ]:
seasonality_components_df = pd.DataFrame({
     "name": ["tod", "tow", "conti_year"],           # component value column name used to create seasonality component
     "period": [24.0, 7.0, 1.0],                     # period for seasonality component
     "order": [3, 3, 5],                             # Fourier series order
     "seas_names": ["daily", "weekly", "yearly"]})   # seasonality component name

res = model.find_seasonality_changepoints(
     df=df,            # data df
     time_col="DIA",    # time column name
     value_col="POSITIVOS_DIA",
     regularization_strength=0.4,                    # between 0.0 and 1.0, greater values imply fewer changepoints, and 1.0 implies no changepoints
     no_changepoint_proportion_from_end=0.2,         # no changepoint in the last 20% data
     trend_changepoints=None,
     seasonality_components_df =seasonality_components_df)                        # optionally specify trend changepoints to avoid calling find_trend_changepoints)  
fig = model.plot(
     seasonality_change=True,                # detected seasonality change points, discussed in next section
     seasonality_change_by_component=True,   # plot seasonality by component (daily, weekly, etc.), discussed in next section
     seasonality_estimate=True,              # plot estimated trend+seasonality, discussed in next section
     plot=False)  
plotly.io.show(fig)
fig.show(renderer="colab")

## Forecasting with changepoints

In [ ]:
# specify dataset information
metadata = dict(
    time_col="DIA",  # name of the time column ("datepartition" in example above)
    value_col="POSITIVOS_DIA",  # name of the value column ("macrosessions" in example above)
    freq="D"        # "H" for hourly, "D" for daily, "W" for weekly, etc.
    # Any format accepted by ``pd.date_range``
)
# specify changepoint parameters in model_components
model_components = dict(
    changepoints={
        # it's ok to provide one of ``changepoints_dict`` or ``seasonality_changepoints_dict`` by itself
        "changepoints_dict": {
            "method": "auto",
            #"yearly_seasonality_order": 15,
            "regularization_strength": 0.5,
            "resample_freq": "2D",
            "potential_changepoint_n": 5,
            "no_changepoint_proportion_from_end": 0.2
        },
        "seasonality_changepoints_dict": {
            "potential_changepoint_distance": "2D",
            "regularization_strength": 0.5,
            "no_changepoint_proportion_from_end": 0.2
        }
    },
    custom={
        "fit_algorithm_dict": {
            "fit_algorithm": "ridge"}})  # use ridge to prevent overfitting when there many changepoints

# Generates model config
config = ForecastConfig.from_dict(
    dict(
        model_template=ModelTemplateEnum.SILVERKITE.name,
        forecast_horizon=31,  # forecast 1 year
        coverage=0.95,  # 95% prediction intervals
        metadata_param=metadata,
        model_components_param=model_components))

# Then run with changepoint parameters
forecaster = Forecaster()
result = forecaster.run_forecast_config(
    df=df,
    config=config)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


In [ ]:
# simple way
#forecaster = Forecaster()  # Creates forecasts and stores the result
#result = forecaster.run_forecast_config(  # result is also stored as `forecaster.forecast_result`.
#     df=df,
#     config=ForecastConfig(
#         model_template=ModelTemplateEnum.SILVERKITE.name,# choose the model template 
#         forecast_horizon=31,  # forecasts steps ahead
#         coverage=0.95,         # 95% prediction intervals
#         metadata_param=metadata,
#     )
# )

In [ ]:
result

ForecastResult(timeseries=<greykite.framework.input.univariate_time_series.UnivariateTimeSeries object at 0x7f2af53f6810>, grid_search=RandomizedSearchCV(cv=RollingTimeSeriesSplit(expanding_window=True, forecast_horizon=31,
            max_splits=3, min_train_periods=62, periods_between_splits=31,
            periods_between_train_test=0, use_most_recent_splits=False),
                   estimator=Pipeline(steps=[('input',
                                              PandasFeatureUnion(transformer_list=[('date',
                                                                                    Pipeline(steps=[('select_date',
                                                                                                     ColumnSelector(column_names=['ts'...
                            'OutsideTolerance3p': make_scorer(score_func_finite),
                            'OutsideTolerance4p': make_scorer(score_func_finite),
                            'OutsideTolerance5p': make_scorer(s

In [ ]:
grid_search = result.grid_search
cv_results = summarize_grid_search_results(
     grid_search=grid_search,
     decimals=2,
     # The below saves space in the printed output. Remove to show all available metrics and columns.
     cv_report_metrics=None,
     column_order=["rank", "mean_test", "split_test", "mean_train", "split_train", "mean_fit_time", "mean_score_time", "params"])
# Transposes to save space in the printed output
cv_results["params"] = cv_results["params"].astype(str)
cv_results.set_index("params", drop=True, inplace=True)
cv_results.transpose()

params,[]
rank_test_MAPE,1
mean_test_MAPE,190.94
split_test_MAPE,"(43.64, 435.42, 93.75)"
mean_train_MAPE,301.2
split_train_MAPE,"(16.93, 433.4, 453.26)"
mean_fit_time,95.86
mean_score_time,1.03


In [ ]:
backtest = result.backtest
fig = backtest.plot()
plotly.io.show(fig)
fig.show(renderer="colab")

In [ ]:
backtest_eval = defaultdict(list)
for metric, value in backtest.train_evaluation.items():
     backtest_eval[metric].append(value)
     backtest_eval[metric].append(backtest.test_evaluation[metric])
metrics = pd.DataFrame(backtest_eval, index=["train", "test"]).T
metrics

,train,test
CORR,0.54087,0.479393
R2,0.290647,-0.554707
MSE,6648.63,29815.4
RMSE,81.5392,172.671
MAE,64.6399,132.955
MedAE,52.8752,105.264
MAPE,390.294,35.6115
MedAPE,63.2768,38.1855
sMAPE,40.4518,22.0241
Q80,33.4627,101.64


In [ ]:
forecast = result.forecast
fig = forecast.plot()
plotly.io.show(fig)
fig.show(renderer="colab")

In [ ]:
forecast.df.head().round(2)

,DIA,actual,forecast,forecast_lower,forecast_upper
0,2020-01-05,1.0,-25.90,-147.74,95.94
1,2020-01-06,NaN,-21.13,-191.68,149.43
2,2020-01-07,NaN,-6.41,-188.77,175.96
3,2020-01-08,NaN,9.52,-151.90,170.95
4,2020-01-09,NaN,-0.61,-183.62,182.40


In [ ]:
fig = forecast.plot_components()
plotly.io.show(fig) 
fig.show(renderer="colab")

In [ ]:
summary = result.model[-1].summary()  # -1 retrieves the estimator from the pipeline
print(summary)

================================ Model Summary =================================

Number of observations: 505,   Number of features: 92
Method: Ridge regression
Number of nonzero features: 92
Regularization parameter: 1.123

Residuals:
         Min           1Q       Median           3Q          Max
      -176.3       -50.27       -7.663        35.04        307.8

             Pred_col Estimate Std. Err Pr(>)_boot sig. code             95%CI
            Intercept    -16.6    6.594      0.008        **  (-28.13, -3.162)
  events_C...New Year   -60.16    34.83      0.026         *      (-111.9, 0.)
  events_C...w Year-1   -34.84     20.6      0.052         .      (-70.32, 0.)
  events_C...w Year-2   -17.67    16.03      0.202             (-48.61, 8.507)
  events_C...w Year+1   -41.66     24.6      0.046         *      (-81.99, 0.)
  events_C...w Year+2   -35.46    24.23      0.102             (-74.38, 5.476)
 events_Christmas Day    39.58    26.23      0.072         .       (0., 78.85)
 

In [ ]:
 model = result.model
 model

Pipeline(steps=[('input',
                 PandasFeatureUnion(transformer_list=[('date',
                                                       Pipeline(steps=[('select_date',
                                                                        ColumnSelector(column_names=['ts']))])),
                                                      ('response',
                                                       Pipeline(steps=[('select_val',
                                                                        ColumnSelector(column_names=['y'])),
                                                                       ('outlier',
                                                                        ZscoreOutlierTransformer()),
                                                                       ('null',
                                                                        NullTransformer(impute_algorithm='interpolate',
                                                                 

In [ ]:
future_df = result.timeseries.make_future_dataframe(
     periods=15,
     include_history=False)

In [ ]:
future_df

,ts,y
2021-05-24,2021-05-24,NaN
2021-05-25,2021-05-25,NaN
2021-05-26,2021-05-26,NaN
2021-05-27,2021-05-27,NaN
2021-05-28,2021-05-28,NaN
2021-05-29,2021-05-29,NaN
2021-05-30,2021-05-30,NaN
2021-05-31,2021-05-31,NaN
2021-06-01,2021-06-01,NaN
2021-06-02,2021-06-02,NaN


In [ ]:
model.predict(future_df)

,ts,forecast,forecast_lower,forecast_upper,y_quantile_summary
0,2021-05-24,285.503872,114.947266,456.060478,"(114.94726553783525, 456.0604778330006)"
1,2021-05-25,263.271862,80.904119,445.639605,"(80.90411858838334, 445.63960493053753)"
2,2021-05-26,221.140147,59.715765,382.564530,"(59.71576517670843, 382.56452957412165)"
3,2021-05-27,243.473544,60.460044,426.487045,"(60.460043587684794, 426.487045329681)"
4,2021-05-28,216.591379,54.249063,378.933695,"(54.24906348469591, 378.9336954005012)"
5,2021-05-29,141.102572,-1.211641,283.416785,"(-1.2116414388160877, 283.41678485480736)"
6,2021-05-30,129.141472,7.303639,250.979306,"(7.303639000344418, 250.97930554796173)"
7,2021-05-31,248.396176,77.839570,418.952782,"(77.83956957253679, 418.9527818677022)"
8,2021-06-01,230.833987,48.466244,413.201730,"(48.4662438470734, 413.20173018922753)"
9,2021-06-02,203.800130,42.375747,365.224512,"(42.37574738923499, 365.22451178664824)"


## Tunning

In [ ]:
from greykite.framework.templates.autogen.forecast_config import EvaluationPeriodParam
from greykite.framework.utils.result_summary import summarize_grid_search_results

In [ ]:
# Defines the cross-validation config
evaluation_period = EvaluationPeriodParam(
    test_horizon=30,             # leaves x days as testing data
    cv_horizon=30,               # each cv test size is x days (same as forecast horizon)
    cv_max_splits=3,              # x folds cv
    cv_min_train_periods=365   # uses at least x years for training
)

# Runs the forecast
result = forecaster.run_forecast_config(
    df=df,
    config=ForecastConfig(
        model_template=ModelTemplateEnum.SILVERKITE.name,
        forecast_horizon=30,  # forecasts x steps ahead
        coverage=0.95,  # 95% prediction intervals
        metadata_param=metadata,
        evaluation_period_param=evaluation_period
    )
)

# Summarizes the cv result
cv_results = summarize_grid_search_results(
    grid_search=result.grid_search,
    decimals=1,
    # The below saves space in the printed output. Remove to show all available metrics and columns.
    cv_report_metrics=None,
    column_order=["rank", "mean_test", "split_test", "mean_train", "split_train", "mean_fit_time", "mean_score_time", "params"])
# Transposes to save space in the printed output
cv_results["params"] = cv_results["params"].astype(str)
cv_results.set_index("params", drop=True, inplace=True)
cv_results.transpose()

Fitting 3 folds for each of 1 candidates, totalling 3 fits


params,[]
rank_test_MAPE,1
mean_test_MAPE,259.3
split_test_MAPE,"(192.7, 451.4, 133.8)"
mean_train_MAPE,417.4
split_train_MAPE,"(436.4, 400.7, 415.1)"
mean_fit_time,9.2
mean_score_time,0.9


In [ ]:
backtest = result.backtest
fig = backtest.plot()
plotly.io.show(fig)
fig.show(renderer="colab")

## anomalies

In [ ]:
fig = ts.plot_quantiles_and_overlays(
     groupby_time_feature="month_dom",
     show_mean=True,
     show_quantiles=False,
     show_overlays=True,
     overlay_label_time_feature="year",
     overlay_style={"line": {"width": 1}, "opacity": 0.5},
     center_values=True,
     xlabel="day of year",
     ylabel=ts.original_value_col,
     title="yearly seasonality for each year (centered)",
 )
 plotly.io.show(fig)